<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E5%88%A9%E7%94%A8%E5%9C%96%E5%BA%AB%E6%89%BE%E5%87%BA%E6%97%A9%E6%9C%9F%E7%99%8C%E7%97%87kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

架構圖註解
以下是程式碼所使用的模型架構圖的註解：

輸入層: 接收大小為 (224, 224, 3) 的影像數據。
ResNet50 基礎模型: 使用預先訓練好的 ResNet50 模型提取影像特徵。
包含多個卷積層、池化層和殘差塊。
這些層的權重在訓練過程中保持不變 (trainable = False)。
Flatten 層: 將基礎模型的輸出展平為一維向量。
Dense 層 (512 個神經元): 全連接層，使用 ReLU 激活函數。
Dropout 層 (0.5): 隨機丟棄 50% 的神經元，防止過擬合。
輸出層 (1 個神經元): 全連接層，使用 sigmoid 激活函數，輸出癌症的機率 (介於 0 到 1 之間)。
希望以上資訊能幫助您理解程式碼和模型架構。

In [ ]:
!pip install nibabel pydicom tensorflow opencv-python Flask

In [ ]:
!pip install nibabel pydicom
import os
import numpy as np
import cv2
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from flask import Flask, request, jsonify

# 數據清洗和預處理
def load_and_preprocess_data(data_dir):
    """
    載入並預處理數據。

    Args:
        data_dir: NIFTI 檔案所在的目錄路徑。

    Returns:
        images: 預處理後的影像數據，形狀為 (num_samples, 224, 224, 1)。
        labels: 對應的標籤數據，形狀為 (num_samples,)。
    """
    images = []
    labels = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(('.nii', '.nii.gz')):  # Check for NIFTI extensions
                nii_path = os.path.join(root, file)
                # 顯示處理中的檔案路徑，方便除錯
                print("處理中:", nii_path)
                img_data = nib.load(nii_path).get_fdata()  # Load NIFTI data
                # Assuming img_data has 3 dimensions (x, y, z) or 4 (x, y, z, time)
                # Select a relevant slice or average across slices/time as needed
                # Example: selecting the middle slice along the z-axis
                img = img_data[:, :, img_data.shape[2] // 2]
                img = cv2.resize(img, (224, 224))
                img = img / np.max(img)  # Normalize to 0-1
                images.append(img)
                # 假設標籤在文件名中
                label = 1 if 'cancer' in file else 0  # Update if needed
                labels.append(label)

    # 檢查是否有載入任何影像
    if not images:
        raise ValueError(f"在 {data_dir} 中找不到任何 NIFTI 檔案。請檢查目錄路徑。")
    return np.array(images)[:, :, :, np.newaxis], np.array(labels)  # Add channel dimension

# 訓練模型
def train_model(X, y):
    """
    訓練癌症偵測模型。

    Args:
        X: 輸入影像數據。
        y: 對應的標籤數據。

    Returns:
        model: 訓練好的模型。
    """
    # 使用 ResNet50 作為基礎模型, 修改輸入形狀以配合 MRI 數據
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 1))
    # 添加自定義層
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # 建立模型
    model = Model(inputs=base_model.input, outputs=predictions)

    # 凍結基礎模型的層
    for layer in base_model.layers:
        layer.trainable = False

    # 編譯模型
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    # 訓練模型
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
    model.save('cancer_detection_model.h5')
    return model

# 部署模型
app = Flask(__name__)
model = None  # Initialize model as None

@app.route('/predict', methods=['POST'])
def predict():
    """
    使用模型進行預測。

    Args:
        file: 上傳的影像檔案。

    Returns:
        prediction: 預測結果，介於 0 到 1 之間，表示癌症的機率。
    """
    global model
    if model is None:  # Load model only once
        model = tf.keras.models.load_model('cancer_detection_model.h5')

    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400  # Handle missing file

    file = request.files['file']

    try:
        # 修改此處以處理 NIFTI 檔案
        img_data = nib.load(file).get_fdata()
        img = img_data[:, :, img_data.shape[2] // 2]  # Assuming 3D data
        img = cv2.resize(img, (224, 224))
        img = img / np.max(img)
        img = np.expand_dims(img, axis=0)
        img = np.expand_dims(img, axis=-1)
        prediction = model.predict(img)
        return jsonify({'prediction': float(prediction[0][0])})
    except Exception as e:
        return jsonify({'error': str(e)}), 500  # Handle prediction errors


if __name__ == '__main__':
    # 將 'path_to_mri_files' 替換為您的 MRI 檔案所在的實際路徑
    data_dir = 'path_to_mri_files'  # 請更新為您的實際路徑
    X, y = load_and_preprocess_data(data_dir)
    train_model(X, y)
    app.run(debug=True, host='0.0.0.0', port=5000)  # Specify host and port